In [1]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 51.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 8.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 21.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 19.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
# Set the Hugging Face token
os.environ["HF_TOKEN"] = ""

In [6]:
# Load the model and tokenizer
model_name = "meta-llama/Meta-Llama-3.1-8B"
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=os.getenv("HF_TOKEN"))
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=os.getenv("HF_TOKEN"))


/usr/local/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

/usr/local/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [7]:
# Assign the eos_token as the pad_token (if needed)
tokenizer.pad_token = tokenizer.eos_token

In [8]:
# Load the dataset
dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")["train"]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

In [9]:
# Create a TF-IDF vectorizer and fit it on the dataset's 'Answer' column
vectorizer = TfidfVectorizer().fit(dataset["Answer"])

def retrieve_documents(question, dataset, vectorizer, top_n=5):
    # Vectorize the question
    question_vector = vectorizer.transform([question])

    # Compute similarity between the question and all answers in the dataset
    similarities = cosine_similarity(question_vector, vectorizer.transform(dataset["Answer"]))

    # Get the indices of the top_n most similar answers
    top_indices = similarities.argsort()[0][-top_n:]
    
    # Convert numpy.int64 indices to regular int
    top_indices = [int(i) for i in top_indices]

    # Retrieve the corresponding answers
    retrieved_docs = [dataset[i]["Answer"] for i in top_indices]
    return " ".join(retrieved_docs)

In [18]:
def rag_pipeline(question, dataset, vectorizer, model, tokenizer):
    # Retrieve relevant documents based on the question
    docs = retrieve_documents(question, dataset, vectorizer)

    # Concatenate the question with the retrieved documents
    combined_input = question + " " + docs

    # Tokenize the combined input
    inputs = tokenizer(combined_input, return_tensors="pt", padding=True, truncation=True, max_length=256)

    # Generate an answer using the model
    outputs = model.generate(
        inputs["input_ids"], 
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50
    )

    # Decode and return the generated answer
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [20]:
# Example usage
question = "Who is at risk for Lymphocytic Choriomeningitis (LCM)?"
answer = rag_pipeline(question, dataset, vectorizer, model, tokenizer)
print("Answer:", answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: Who is at risk for Lymphocytic Choriomeningitis (LCM)? Lymphocytic colitis is form of microscopic colitis, a condition that is characterized by inflammation of the colon (large intestines). As the name suggests, microscopic colitis can only be diagnosed by examining a small sample of colon tissue under a microscope. In lymphocytic colitis, specifically, the tissues and lining of the colon are of normal thickness, but an increase in the number of lymphocytes (a type of white blood cell) is observed. Signs and symptoms of the condition may include chronic, watery diarrhea; abdominal pain, cramping, and bloating; weight loss; nausea; dehydration; and/or fecal incontinence. The underlying cause of lymphocytic colitis is currently unknown; however, scientists suspect that autoimmune conditions, medications, infections, genetic factors, and/or bile acid malabsorption may contribute to the development of the condition. Treatment is based on the signs and symptoms present in each perso

In [21]:
# Example usage
question = "How to diagnose Parasites - Cysticercosis ?"
answer = rag_pipeline(question, dataset, vectorizer, model, tokenizer)
print("Answer:", answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: How to diagnose Parasites - Cysticercosis? Cysticercosis is an infection caused by the larvae of the parasite Taenia solium.  This infection occurs after a person swallows tapeworm eggs. The larvae get into tissues such as muscle and brain, and form cysts there (these are called cysticerci). When cysts are found in the brain, the condition is called neurocysticercosis. Diagnosis of Taenia tapeworm infections is made by examination of stool samples; individuals should also be asked if they have passed tapeworm segments. Stool specimens should be collected on three different days and examined in the lab for Taenia eggs using a microscope. Tapeworm eggs can be detected in the stool 2 to 3 months after the tapeworm infection is established.
    
Tapeworm eggs of T. solium can also infect humans, causing cysticercosis. It is important to diagnose and treat all tapeworm infections.
    
More on: cysticercosis If you think that you may have cysticercosis, please see your health care p